In [1]:
import os
os.getcwd() 

'/Users/izapreev/Projects/ML-PT'

In [2]:
# Make sure the source code auto reloads into the kernel
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

from src.utils.logger import logger

In [4]:
from src.utils.file_utils import load_numpy_zc

# Load the features file for further use
file_name = os.path.join('.', 'data', 'X.npz')
X, = load_numpy_zc(file_name, ['X'])

15:40:01 INFO (file_utils:32): Loading compressed numpy z file from: ./data/X.npz
15:40:01 INFO (file_utils:35): File loading and data extraction are done!


In [5]:
# Load the classes file for further use
file_name = os.path.join('.', 'data', 'y.npz')
y, = load_numpy_zc(file_name, ['y'])

15:40:01 INFO (file_utils:32): Loading compressed numpy z file from: ./data/y.npz
15:40:01 INFO (file_utils:35): File loading and data extraction are done!


In [6]:
#########################################################################
# 01 Perform initial hyperparameters tuning for Random Forest Classifier
#

In [9]:
# Define the main hyperparameters to tune
num_features = X.shape[1]
num_classes  = len(np.unique(y))
random_grid = {
                'n_estimators' : [20, 30, 40, 50, 60, 70, 80],
                'criterion' : ['gini', 'entropy', 'log_loss'],
                'max_depth' : [None, 10, 20, 30, 40, 50],
                'min_samples_leaf' : [1, 2, 4, 6],
                'min_samples_split' : [2, 4, 6, 8],
                'max_features' : list(range(1, num_features + 1)),
                'max_leaf_nodes' : [None, num_classes, 2 * num_classes, 3 * num_classes]
              }

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available 
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)

# Fit the random search model. NOTE: Here we use non-scaled/shifter inputs as for the RandomForestClassifier it is not necessary
logger.info(f'Start the Random Grid model tuning')
rf_random.fit(X, y)

# Log the best parameters found
logger.info(f'Found the best Random search parameters: {rf_random.best_params_}')

15:42:16 INFO (3191410847:10): Start the Random Grid model tuning


Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
# TODO: Do the crude grid search

In [ ]:
# TODO: Do the fine grid search

In [ ]:
# TODO: Train the resulting model

In [ ]:
# TODO: Store the trained classification model